# Description

Training a Multi-Layer Perceptron for genre classification from the data extracted using preprocessor.

# Packages

In [ ]:
import json
import numpy as np
from  sklearn.model_selection import train_test_split
import tensorflow as tf

# Loading data

In [3]:
DATA_PATH = "preprocessed_raw_audio_data.json"

In [1]:
def load_data(path):

    with open(path, "r") as fp:
        data = json.load(fp)
    
    features = np.array(data["mfcc"])
    target = np.array(data["labels"])

    return features, target


In [4]:
features, target = load_data(DATA_PATH)

In [13]:
# try
print(f"Feature: {features[0]} \n Label: {target[0]}")

Feature: [[-2.42435989e+02  1.02184868e+02 -9.91323471e+00 ... -3.95323277e+00
  -2.40564632e+00 -2.38090801e+00]
 [-2.12989746e+02  1.03544785e+02 -1.08341713e+01 ... -5.81353378e+00
   2.64356709e+00 -2.80662346e+00]
 [-1.95409302e+02  1.04360657e+02 -1.85502887e-01 ... -1.08171625e+01
  -5.88805974e-01 -5.28410244e+00]
 ...
 [-1.24157700e+02  6.37352219e+01  2.11729393e+01 ... -1.65676632e+01
   1.37965145e+01 -2.31835346e+01]
 [-1.11283936e+02  6.05447235e+01  1.33710217e+00 ... -1.22842150e+01
   1.48256063e+01 -1.58052387e+01]
 [-1.31811951e+02  6.65644531e+01 -7.93185902e+00 ... -8.66252232e+00
   1.16732750e+01 -8.72385406e+00]] 
 Label: 0


In [14]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3)

# Model definiton

In [9]:
model = tf.keras.Sequential([
    
     tf.keras.layers.Flatten(input_shape=(features.shape[1], features.shape[2])), # since feature is a 3d array

     tf.keras.layers.Dense(units=512, activation="relu"),

     tf.keras.layers.Dense(units=256, activation="relu"),

     tf.keras.layers.Dense(units=64, activation="relu"),

     tf.keras.layers.Dense(units=10, activation='linear')
  
])

c:\Users\acer\Desktop\Projects\Music Genre Classifier\venv\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

In [11]:
model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics =["accuracy"])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 1690)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       865,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,014,218 (3.87 MB)

 Trainable params: 1,014,218 (3.87 MB)

 Non-trainable params: 0 (0.00 B)

# Training

In [15]:
model.fit(X_train, y_train, epochs=50, batch_size=32)

Epoch 1/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.2377 - loss: 14.6147
Epoch 2/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.3428 - loss: 2.0209
Epoch 3/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.3783 - loss: 1.7853
Epoch 4/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.4014 - loss: 1.7388
Epoch 5/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.4487 - loss: 1.5848
Epoch 6/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.4571 - loss: 1.5643
Epoch 7/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.4884 - loss: 1.4920
Epoch 8/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5013 - loss: 1.4439
Epoch 9/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.5230 - loss: 1.3970
Epoch 10/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.5387 - loss: 1.3419
Epoch 11/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.5655 - loss: 1.2499
Epoch 12/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/ste

# Evaluation

In [35]:
logits = model.predict(X_test)
probabilities = tf.nn.softmax(logits)
predicted_genres = tf.argmax(probabilities, axis=-1).numpy()

accuracy = np.mean(y_test==predicted_genres)*100

print(f"Accuracy: {accuracy:.4f}")


94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 57.3097
